In [ ]:
import sys
SAVE = "/saved/"    #In this folder model snapshot are saved
LIB = "/"           #This contains the IIC-SimCLR implementation files
DATA = "/datasets/MNIST/" #In this folder datasets will be downloaded
sys.path.append(LIB)

In [ ]:
import wandb
import torch
from collections import namedtuple
from torchvision.datasets import CIFAR10
from torchvision.datasets import STL10
from torchvision.datasets import MNIST
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from simclr_clustering import *
from simclr_clustering.data.transforms import *
from simclr_clustering.data.cluster_eval import *
from simclr_clustering.data.eval_metrics import *
from simclr_clustering.data.make_data import *
from simclr_clustering.archs import ClusterNetMulHead
from simclr_clustering.archs import IICLoss, NTXentLoss
from simclr_clustering.pipeline import IIC_Pipeline

In [ ]:
config = {
    "output_k_A": 70,
    "output_k_B": 10,
    "output_k_C": 128,
    "output_back": 512,
    "gt_k": 10, #grond truth classes
    "separate_augs" : False;
    "input_sz": 32,
    "rand_crop_sz": 20,
    "rot_val": 0,
    "fluid_warp": False,
    "crop_orig": True,
    "multiple_crop": False,
    "crop_szs": [16,20,24],
    "cutout": False,
    "sobel": True,
    "cutout_p": False,
    "cutout_max_box": False,
    "demean": False,
    "per_img_demean": False,
    "simclr_bs": 512,
    "iic_dataloader_bs": 220, # iic_bs / num_dataloaders
    "iic_bs": 660,
    "batchnorm_track" : True,
    "include_rgb" : False,
    "using_ir" : False,
    "in_channels" : 2,
    "num_sub_heads" : 5,
    "num_dataloaders": 3,
    "dataset": 'CIFAR10',
    "dataset_root": DATA,
    "train_partitions_head_A" : [True, False],
    "train_partitions_head_B" : [True, False],
    "mapping_assignment_partitions" : [True, False],
    "mapping_test_partitions" : [True, False],
    "eval_mode": 'hung', #orig for original, hung for hungarian
    "test_code": False,
    "num_epochs": 500,
    "device": 'cuda',
    "warmup": 20,
    "lr_iic": 0.0001,
    "lr_simclr": 0.001,
    "lr_gamma_iic": None,
    "lr_milestones_iic": None,
    "num_epochs_head": 1,
}

config_log = {
    'log' : False,
    'project' : 'my-project',
    'run': 'my-run'
}
config_log = namedtuple('Config_l',config_log.keys())(*config_log.values())
config = namedtuple('Config',config.keys())(*config.values())

In [ ]:
_ = CIFAR10(config.dataset_root, train=True, download=True)
_ = CIFAR10(config.dataset_root, train=False, download=True)

iic = IIC_Pipeline(config)
#uncomment next line to load a previously saved checkpoint
#iic.load_checkpoint(path=SAVE+'model.pth')
iic.exec_pipeline(config_log, save_path=SAVE+'model.pth', do_clustering=True, do_overclustering=True, do_simclr=True)